# Image Data Base Creation 

### 1-Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
from PIL import Image
import matplotlib.pyplot as plt
from pymongo import MongoClient
import blosc


### 2 - Data Frame

Has been created a dataframe with the most representative buildings of each style in a CSV, wich is going to be load in Notebook. The CSV has the link to the pictures of the buildings, which should be transformed into another format to upload the data in Mongo BD

In [ ]:
df_building=pd.read_csv("./examples/Buildings.csv", sep= ";",header=0, names=["Number","School","Name","Architects","Years","Link"])

In [ ]:
df_building

### 3- Test -- to check BLOSC extension performance

In [ ]:
picture=Image.open(df_building["Link"][2]).convert("RGB")
picture.show()

compressed_b = blosc.pack_array(np.array(picture))
print(compressed_b)
print(type(compressed_b))

In [ ]:
decompressed_array = blosc.unpack_array(compressed_b)
im = Image.fromarray(decompressed_array)

im


### 4- Reshape and transform all the pictures into BLOSC files and added into the Data Frame

In [ ]:
pict=[]
dimension=[]

for i in df_building["Link"]:
    im_compress=0
    picture=Image.open(i).convert('RGB')

    if picture.size[0]<picture.size[1]:
        picture=picture.resize((200,160))
        dimension.append(picture.size)
        im_compress = blosc.pack_array(np.array(picture))
        
    else:
        picture=picture.resize((200,250))
        dimension.append(picture.size)
        im_compress = blosc.pack_array(np.array(picture))
    
    pict.append(im_compress)

print(dimension)
print(len(pict))

In [ ]:
df_building=df_building.assign(Pictures=pict)
df_building=df_building.assign(Dimension=dimension)

In [ ]:
df_building=df_building.drop(columns="Link")
df_building.head()

In [ ]:
#picture=Image.open(df_building["Link"][2]).convert("RGB")

decompressed_array = blosc.unpack_array(df_building["Pictures"][1])
print(type(decompressed_array))
print(df_building["Pictures"][1])
im = Image.fromarray(decompressed_array)

im


### 5- MongoDB Connection and create the DataBase

In [ ]:
##mongo_url= ADD HERE YOUR MONGO URL 

client = MongoClient(mongo_url)
db = client.get_database("Data")
match_db= db.Styles

In [ ]:
client

In [ ]:
for row in df_building.index:
    edificio={}
    for column in df_building:
        if column=="Number":
            pass
        else:
            if column=="Years":
                edificio[column]=int(df_building[column][row])
            else:
                edificio[column]=df_building[column][row]
    match_db.insert_one(edificio)    
    print(edificio["Name"])

In [ ]:
##mongo_url= ADD HERE YOUR MONGO URL 

client = MongoClient(mongo_url)
db = client.get_database("Data")
match_db= db.Styles

In [ ]:
res=list(db["Styles"].find({"Name":'Auditorium Building'}))

print(res[0]['Pictures'])

cosa=res[0]['Pictures']

In [ ]:
decompressed_array = blosc.unpack_array(cosa)
print(type(decompressed_array))
print(type(cosa))
im = Image.fromarray(decompressed_array)

im
